# Databricks Markdown Notebook: PySpark Oefeningen voor Data Engineers

## 1. Oefening: Creëer een DataFrame

In [ ]:
from pyspark.sql import SparkSession

# SparkSession aanmaken
spark = SparkSession.builder.appName("PySparkOefeningen").getOrCreate()

# Dataset maken
patienten_data = [("Jan", 45, "Basis"), ("Marie", 55, "Uitgebreid"), ("Karin", 60, "Basis")]
kolommen = ["Naam", "Leeftijd", "Verzekering"]

# DataFrame maken
df = spark.createDataFrame(patienten_data, kolommen)
df.show()


## 2. Oefening: Filteren en Selecteren

In [ ]:
from pyspark.sql.functions import col

# Selecteer alleen de patiënten met een leeftijd boven 50
oudere_patienten = df.filter(col("Leeftijd") > 50)
oudere_patienten.show()


## 3. Oefening: Kolommen Bewerken

In [ ]:
# Een nieuwe kolom toevoegen die aangeeft of iemand senior is
df = df.withColumn("Senior", col("Leeftijd") >= 60)
df.show()


## 4. Oefening: Groeperen en Aggregaties

In [ ]:
# Tel het aantal patiënten per verzekeringssoort
df.groupBy("Verzekering").count().show()


## 5. Oefening: Joins

In [ ]:
# Extra dataset maken met medische kosten
kosten_data = [("Jan", 200), ("Marie", 350), ("Karin", 150)]
kosten_kolommen = ["Naam", "Kosten"]
kosten_df = spark.createDataFrame(kosten_data, kosten_kolommen)

# Join uitvoeren op Naam
df_kosten = df.join(kosten_df, "Naam", "inner")
df_kosten.show()


## 6. Oefening: Werken met Missing Values

In [ ]:
from pyspark.sql.functions import when

# Een nieuwe dataset met ontbrekende waarden
data_missend = [("Jan", 45, "Basis"), ("Marie", None, "Uitgebreid"), ("Karin", 60, None)]
df_missend = spark.createDataFrame(data_missend, kolommen)

# Vervang ontbrekende waarden in de kolom "Leeftijd" door het gemiddelde
mean_leeftijd = df_missend.selectExpr("avg(Leeftijd) as mean_leeftijd").collect()[0]["mean_leeftijd"]
df_missend = df_missend.fillna({"Leeftijd": mean_leeftijd, "Verzekering": "Onbekend"})
df_missend.show()


## 7. Oefening: Sorting en Ordering

In [ ]:
# Sorteer de patiënten op leeftijd
df_sorted = df.orderBy(col("Leeftijd").desc())
df_sorted.show()


## 8. Oefening: UDF (User Defined Function)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Een UDF om een leeftijdscategorie toe te voegen
def leeftijd_categorie(leeftijd):
    if leeftijd < 50:
        return "Jongvolwassene"
    elif 50 <= leeftijd < 65:
        return "Volwassene"
    else:
        return "Senior"

leeftijd_categorie_udf = udf(leeftijd_categorie, StringType())

df = df.withColumn("Leeftijdscategorie", leeftijd_categorie_udf(col("Leeftijd")))
df.show()


## 9. Oefening: Partitioneren en Caching

In [ ]:
# Partitioneer de dataset op verzekeringssoort en cache het resultaat
df_partitioned = df.repartition(2, "Verzekering")
df_partitioned.cache()
df_partitioned.show()


## 10. Oefening: Data Opslaan

In [ ]:
# Data opslaan als Parquet-bestand
df.write.mode("overwrite").parquet("/mnt/data/patienten.parquet")


## 11. Oefening: Data Lezen

In [ ]:
# Data opnieuw inlezen vanuit Parquet
df_lezen = spark.read.parquet("/mnt/data/patienten.parquet")
df_lezen.show()


## 12. Oefening: Window Functions

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Maak een Window om rijen te nummeren per verzekering
window_spec = Window.partitionBy("Verzekering").orderBy(col("Leeftijd"))
df = df.withColumn("RijNummer", row_number().over(window_spec))
df.show()


## 13. Oefening: JSON Data Lezen

In [ ]:
# JSON-bestand lezen (voorbeeld)
df_json = spark.read.json("/mnt/data/patienten.json")
df_json.show()


## 14. Oefening: CSV Data Lezen

In [ ]:
# CSV-bestand lezen (voorbeeld)
df_csv = spark.read.option("header", "true").csv("/mnt/data/patienten.csv")
df_csv.show()


## 15. Oefening: DataFrame naar Pandas Converteren

In [ ]:
# Omzetten naar Pandas DataFrame
pandas_df = df.toPandas()
print(pandas_df.head())
